In [1]:
import numpy as np

from IPython.display import clear_output
import time

In [2]:
class Environment:
    def __init__(self, jumps, board_size):
        self.__board_size = board_size
        self.__jumps = jumps
        self.__x = board_size // 2
        self.__y = board_size // 2
        
    def reset(self):
        self.__x = 2
        self.__y = 2
        
    def __apply_move(self, x, y, action):
        new_x, new_y = x, y
        
        if action == 'L':
            new_y = y - 1
        elif action == 'R':
            new_y = y + 1
        elif action == 'D':
            new_x = x + 1
        elif action == 'U':
            new_x = x - 1
            
        return new_x, new_y
    
    def __check_jump(self, current_x, current_y):
        for jump in self.__jumps:
            if jump[0][0] == current_x and jump[0][1] == current_y:
                return jump
            
        return None
    
    def __get_new_pos(self, new_x, new_y):
        reward = 0
        
        if 0 > new_x or new_x > self.__board_size - 1:
            new_x = max(min(new_x, self.__board_size - 1), 0)
            reward = -1
        if 0 > new_y or new_y > self.__board_size - 1:
            new_y = max(min(new_y, self.__board_size - 1), 0)
            reward = -1
        
        return new_x, new_y, reward

    
    def predict_reward(self, current_x, current_y, action):
        jump = self.__check_jump(current_x, current_y)
        if jump is not None:
            new_x, new_y, reward = jump[1][0], jump[1][1], jump[2]
        else:
            new_x, new_y = self.__apply_move(current_x, current_y, action)
            new_x, new_y, reward = self.__get_new_pos(new_x, new_y)
        
        return new_x, new_y, reward
    
    def move(self, action):
        jump = self.__check_jump(self.__x, self.__y)
        if jump is not None:
            self.__x, self.__y, reward = jump[1][0], jump[1][1], jump[2]
        else:
            new_x, new_y = self.__apply_move(self.__x, self.__y, action)
            self.__x, self.__y, reward = self.__get_new_pos(new_x, new_y)
        
        return reward
        
    def show(self):
        for i in range(self.__board_size):
            for j in range(self.__board_size):
                if i == self.__x and j == self.__y:
                    print('A', end=' ')
                else:
                    print('_', end=' ')
            print('\n')
    
    def percept(self):
        return self.__x, self.__y
    
            
    @property
    def actions(self):
        return ['U', 'D', 'L', 'R']
    
    @property
    def num_actions(self):
        return 4
    
    @property
    def board_size(self):
        return self.__board_size

In [3]:
def default_env():
    return Environment(
        [
            ((0, 1), (4, 1), 10),
            ((0, 3), (2, 3), 5)
        ],
        5
     )

In [4]:
def choose_an_action(env, state_probs):
    probs = [state_probs[action] for action in env.actions]
    action = np.random.choice(env.actions, p=probs)
    
    return action

In [5]:
def run_episode(env, policy, num_steps, epsilon):
    env.reset()
    
    rewards = []
    states = [env.percept()]
    x, y = states[0]
    
    for _ in range(num_steps):
        probs = policy[x][y]
        
        if np.random.rand() < epsilon:
            action = np.random.choice(env.actions)
        else:
            action = choose_an_action(env, probs)
            
        reward = env.move(action)
        state = env.percept()
        
        states.append(state)
        x, y = states[-1]
        rewards.append(reward)
        
    return states, rewards

In [6]:
# def calculate_value_function(env, policy, discount_factor):
#     values = np.zeros((env.board_size, env.board_size))
    
#     for i in range(10):
#         values = update_values(env, discount_factor, values, policy)

#     return values

In [7]:
def calculate_bellman_equation(env, discount_factor, values, policy, i, j):
    value = 0
    
    for action in env.actions:
        new_x, new_y, reward = env.predict_reward(i, j, action)
        value += policy[i][j][action] * (reward + discount_factor * values[new_x, new_y])
        
    return value

In [8]:
def update_values(env, discount_factor, current_values, policy):
    new_values = np.zeros((env.board_size, env.board_size))
    
    for i in range(env.board_size):
        for j in range(env.board_size):
            new_values[i, j] = calculate_bellman_equation(env, discount_factor, current_values, policy, i, j)
            
    return new_values

In [9]:
def calculate_average_return(states, rewards, discount_factor):
    returns = [0]
    
    for state, reward in zip(reversed(states), reversed(rewards)):
        r = discount_factor * returns[0] + reward
        returns.insert(0, r)
        
    return returns

In [10]:
def first_visit_monte_carlo_update(values, states, returns):
    seen_states = set()
    
    for state, r in zip(states, returns):
        if state not in seen_states:
            values[state[0]][state[1]] = r
            seen_states.add(state)
            
    return values

In [11]:
def flatten_values(env, policy, values):
    policy = [[{action: 0 for action in env.actions} for _ in range(env.board_size)] for _ in range(env.board_size)] 
    
    for i in range(env.board_size):
        for j in range(env.board_size):
            for action in env.actions:
                new_x, new_y, _ = env.predict_reward(i, j, action)
                
                policy[i][j][action] = values[new_x, new_y]
                
    return policy

In [12]:
def multi_max(env, state_values):
    maximum_value = float('-inf')
    maximums = []
    
    for action in env.actions:
        if state_values[action] > maximum_value:
            maximum_value = state_values[action]
            maximums = [action]
        elif state_values[action] == maximum_value:
            maximums.append(action)
            
    return maximums

In [13]:
def update_policy(env, policy):
    new_policy = [[{action: 0 for action in env.actions} for _ in range(env.board_size)] for _ in range(env.board_size)] 
    
    for i in range(env.board_size):
        for j in range(env.board_size):
            optimal_actions = multi_max(env, policy[i][j])
            prob = 1 / len(optimal_actions)
            new_policy[i][j] = {action: prob if action in optimal_actions else 0 for action in env.actions}
    
    return new_policy

In [14]:
def print_policy(env, policy):
    
    for i in range(env.board_size):
        row = []
        for j in range(env.board_size):
            actions = ' '.join(multi_max(env, policy[i][j])).center(10)
            print(actions, end='|')
            
        print()

In [15]:
def first_visit_monte_carlo(env, discount_factor, epsilon, iterations, print_steps=False):
    policy = [[{action: 1/4 for action in env.actions} for _ in range(env.board_size)] for _ in range(env.board_size)]
    values = np.zeros((env.board_size, env.board_size))
    
    for i in range(iterations):
        states, rewards = run_episode(env, policy, 10, epsilon)
        returns = calculate_average_return(states, rewards, discount_factor)
        
        values = values + .2 * first_visit_monte_carlo_update(values, states, returns)
        f_values = flatten_values(env, policy, values)
        policy = update_policy(env, f_values)
        
        if print_steps:
            print_policy(env, policy)
            
            if i != iterations - 1:
                print('-' * 100)
    
    return policy, values

In [16]:
discount_factor = .9
epsilon = .1

In [17]:
env = default_env()

policy = [[{action: 1/4 for action in env.actions} for _ in range(env.board_size)] for _ in range(env.board_size)]

In [18]:
policy, values = first_visit_monte_carlo(env, discount_factor, epsilon, 100, print_steps=True)

 U D L R  | U D L R  | U D L R  | U D L R  | U D L R  |
 U D L R  | U D L R  | U D L R  | U D L R  | U D L R  |
 U D L R  | U D L R  | U D L R  | U D L R  | U D L R  |
 U D L R  | U D L R  | U D L R  | U D L R  | U D L R  |
 U D L R  | U D L R  | U D L R  | U D L R  | U D L R  |
----------------------------------------------------------------------------------------------------
 U D L R  | U D L R  | U D L R  | U D L R  | U D L R  |
  U L R   |  U L R   |  U L R   | U D L R  | U D L R  |
   U D    |    U     |   U R    |   U R    | U D L R  |
   D L    |   D L    |    D     |   U R    |   U R    |
 U D L R  |  D L R   |   D L    |    L     |    U     |
----------------------------------------------------------------------------------------------------
 U D L R  | U D L R  | U D L R  | U D L R  | U D L R  |
  U L R   |  U L R   | U D L R  | U D L R  | U D L R  |
   U D    |   U R    |   U R    |  U L R   | U D L R  |
   D L    |   D L    |   U D    |   U R    |   U R    |
 U D L R  |  D

    R     |    U     |    L     |    U     |  U L R   |
    R     |    U     |    L     |    U     |    U     |
    R     |    U     |    L     |    L     |    L     |
    R     |    U     |    L     |    U     |    U     |
----------------------------------------------------------------------------------------------------
    R     | U D L R  |    L     | U D L R  |    L     |
    R     |    U     |    L     |    U     |    L     |
    R     |    U     |    L     |    U     |    L     |
    R     |    U     |    L     |    U     |    L     |
    R     |    U     |    L     |    U     |    U     |
----------------------------------------------------------------------------------------------------
    R     | U D L R  |    L     | U D L R  |    L     |
    R     |    U     |    L     |    U     |    L     |
    R     |    U     |    L     |    U     |    L     |
    R     |    U     |    U     |    U     |    L     |
    R     |    U     |    L     |    U     |    U     |
--------------

    R     |    U     |    L     |    L     |    L     |
   U R    |    U     |    U     |    U     |    L     |
    R     |   U D    |   U L    |    U     |    U     |
----------------------------------------------------------------------------------------------------
    R     | U D L R  |    L     | U D L R  | U D L R  |
    R     |    U     |    L     |    D     |  U L R   |
    R     |    U     |    R     |    L     |    L     |
   U R    |    U     |    U     |    U     |    L     |
   D L    |    U     |    U     |    U     |    U     |
----------------------------------------------------------------------------------------------------
    R     | U D L R  |    L     | U D L R  | U D L R  |
    R     |    U     |    L     | U D L R  | U D L R  |
    R     |    U     |    L     |  U L R   |  U L R   |
   U R    |    U     |   U L    |   U L    |    U     |
   D L    |    U     |    U     |    U     |    U     |
----------------------------------------------------------------------